In [1]:
%env AIIDA_PATH=/home/jyu/.aiida

%load_ext aiida
%aiida

env: AIIDA_PATH=/home/jyu/.aiida


Loaded AiiDA DB environment - profile name: 2023-08-07.

In [2]:
from aiida import load_profile

load_profile("2023-08-07")

Profile<uuid='05b2711ba5fd4521a4dabced0ec5e128' name='2023-08-07'>

In [10]:
from aiida import orm
# get all nodes of one failed calcjob
n = orm.load_node(239346)
for node in n.base.caching.get_all_same_nodes():
    if node.base.caching.is_created_from_cache:
        cached_from = node.base.extras.get("_aiida_cached_from")
        cached_from_pk = orm.load_node(cached_from).pk
    else:
        cached_from_pk = None
    print(node.pk, node.is_valid_cache, node.base.caching.is_created_from_cache, cached_from_pk)

202152 True True 202139
239346 True True 202152
196453 True False None
239308 True True 202152


In [11]:
def restore_caching_property(node_caching_mapping):
    for pk, is_valid_cache in node_caching_mapping.items():
        node = orm.load_node(pk)
        node.is_valid_cache = is_valid_cache
        print(f"restore node {pk} is_valid_cache={is_valid_cache}")

In [ ]:
# Original caching mapping
node_caching_mapping = {
    "202152": True, # bands
    "239346": True, # pressure
    "196453": True, # original _caching calcjob
    "239308": True, # cohesive energy
    "202139": False, # this is the phonon calcjob that finished and because the source of 202152
}

In [14]:
node_caching_mapping = {
    "202152": False, # bands
    "239346": False, # pressure
    "196453": True, # original _caching calcjob
    "239308": False, # cohesive energy
    "202139": False, # this is the phonon calcjob that finished and because the source of 202152
}

In [15]:
restore_caching_property(node_caching_mapping)

restore node 202152 is_valid_cache=False
restore node 239346 is_valid_cache=False
restore node 196453 is_valid_cache=True
restore node 239308 is_valid_cache=False
restore node 202139 is_valid_cache=False


In [ ]:
# THis is the command to run 
# aiida-sssp-workflow launch --property convergence.bands --pw-code pw-7.0@daint-mc-mrcloud-mem --ph-code ph-7.0@daint-mc-mrcloud-mem --protocol acwf --cutoff-control standard --criteria precision --withmpi True --num-mpiprocs 32 --npool 4 --comment Oxygen --no-daemon -- /home/jyu/Projects/WP-SSSP/sssp-verify-scripts/_sssp_pbe/O/O.us.z_6.ld1.psl.v1.0.0-low.upf
